<a href="https://colab.research.google.com/github/Parsa33033/ADM_HW3/blob/main/homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADM Homework 3

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
import requests
import os
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.request import Request, urlopen
import pandas as pd
import csv
import re
import time

## 1. Data collection

In [3]:
num_of_pages = 384
directory = ""
links_file = "anime.csv"

### 1.1. Get the list of animes

In [ ]:
anime = []

                               
if not os.path.isfile(links_file):
    with open(directory + links_file, "w", encoding='utf-8', newline='') as fobj:
        fobj.write("page,name,url\n")
        for page in tqdm(range(0, num_of_pages)):
            url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
            response = requests.get(url)

            soup = BeautifulSoup(response.text, 'html.parser')
            for tag in soup.find_all('tr'):
                links = tag.find_all('a')
                for link in links:        
                    if type(link.get('id')) == str and len(link.contents[0]) > 1:
                        x = (page + 1, link.contents[0], link.get('href'))
                        anime.append(x)
                        writer = csv.writer(fobj)
                        writer.writerow(x)

100%|██████████| 384/384 [02:53<00:00,  2.21it/s]


### 1.2. Crawl animes

In [4]:
pages_dir = directory + "pages"
page_dir = pages_dir + "/" + "page"
article = "article"

In [ ]:
if not os.path.exists(pages_dir):
    os.makedirs(pages_dir)
    
def urlEncodeNonAscii(b):
    return re.sub('[\x80-\xFF]', lambda c: '%%%02x' % ord(c.group(0)), b)

ds = pd.read_csv(directory + links_file)
i = 1
for p, u in tqdm(zip(ds.page, ds.url)):
    d = page_dir + str(p)
    if not os.path.exists(d):
        os.makedirs(d)
    d = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
    
    if not os.path.exists(d):
#         request = requests.get(u, headers={'Cache-Control': 'no-cache', 'User-Agent': 'Mozilla/5.0'})
        req = Request(u, headers={'User-Agent': 'XYZ/3.0'})
        webpage = urlopen(req, timeout=100).read()
        with open(d, "w", encoding='utf-8') as fobj:
            fobj.write(webpage)
    i += 1

### 1.3 Parse downloaded pages

In [8]:
def getText(parent):
    return ''.join(parent.find_all(text=True, recursive=False)).strip()

In [18]:
def article_parser(article_path, soup):
        anime_title = soup.title.text.strip()
        anime_type = ""
        try:
            anime_type = getText(soup.find('span', text="Type:").parent.a)
        except:
            pass
        anime_num_episodes = ""
        try:
            anime_num_episodes = getText(soup.find('span', text="Episodes:").parent)
        except:
            pass
        anime_aired = ""
        try:
            anime_aired = getText(soup.find('span', text="Aired:").parent).split(" to ")
        except:
            pass
        anime_status = ""
        try:
            anime_status = getText(soup.find('span', text="Status:").parent)
        except:
            pass
        anime_score = ""
        try:
            anime_score = getText(soup.find('span', text="Score:").parent.find_all('span', {'class': 'score-label'})[0])
        except:
            pass
        anime_users = ""
        try:
            anime_users = getText(soup.find('span', text="Members:").parent)
        except:
            pass
        anime_rank = ""
        try:
            anime_rank = getText(soup.find('span', text="Ranked:").parent)
        except:
            pass
        anime_popularity = ""
        try:
            anime_popularity = getText(soup.find('span', text="Popularity:").parent)
        except:
            pass
        anime_description = ""
        try:
            anime_description = getText(soup.find('h2', text="Synopsis").parent.parent.p)
        except:
            pass
        anime_related = []
        try: 
            anime_related = list(set(map(getText,soup.find('h2', text="Related Anime").parent.parent.tr.find_all(lambda tag:tag.name == "a" and tag.href != "")))) 
        except: 
            pass
        anime_characters = []
        try:
            anime_characters = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[1].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
        except:
            pass
        anime_voices = []
        try:
            anime_voices = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[0].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
        except:
            pass
        anime_staff = []
        try:
            anime_staff = list(map(lambda tr: re.split("\n+", tr.text.strip()), soup.find('h2', text="Staff").find_next('div').find_all('tr')))
        except:
            pass
        l = [anime_title,
                anime_type,
                anime_num_episodes,
                anime_aired,
                anime_status,
                anime_score,
                anime_users,
                anime_rank,
                anime_popularity,
                anime_description,
                anime_related,
                anime_characters,
                anime_voices,
                anime_staff]
        return l
    

In [32]:
num_of_docs = 19125
i = 1
for p in range(1, num_of_pages + 1):
    flag = True
    if (flag): 
        article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
    while os.path.isfile(article_path):
        article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
        if os.path.isfile(article_path):
            if i % 100 == 0:
                print(p, "--->", i, "--->", article_path)
                break
            html = open(article_path, 'rb')
            soup = BeautifulSoup(html, "html.parser")
            l = article_parser(article_path, soup)
            if not os.path.exists('docs'):
                os.makedirs('docs')
            anime = open('docs/anime'+str(i)+'.tsv', 'w', encoding="utf-8", newline='')
            writer = csv.writer(anime, delimiter='\t')
            writer.writerow(l)
        i += 1
                


2 ---> 100 ---> pages/page2/article_100.html


## 5. Algorithmic question

 In this problem, the goal is to maximize the sum of values contained in a list, where each value represents the time that the person who made the appointment wants to spend with the personal trainer, so you want to provide the personal trainer with a program that maximizes the total length of accepted appointments. 
The important condition is that the personal trainer needs a break between appointments and cannot accept two consecutive bookings. In addition, reservations are considered in the order in which they were made, so that the personal trainer cannot go back and accept past bookings, but must follow the chronological order. 

Therefore, considering the need for a break between one appointment and another and considering that past appointments cannot be accepted but it is necessary to keep to the chronological order, one solution could be to scroll through the list of bookings twice, that is once starting from the first reservation and then from the index 0, or even, and continuing with step 2 for all even indices and scroll the list a second time starting from the second reservation and then from the index 1, or odd, and proceeding with step 2 for all odd indices. 
Then add the values of the even and odd indices and compare the obtained values. The path that maximizes the sum, will be preferred. 


In [28]:
def appointments (appointments):
    l1 = []
    res1 = 0
    l2 = []
    res2 = 0
    for i in range(len(appointments)):
        if ((i % 2) == 0):
            res1 += appointments[i]
            l1.append(appointments[i])
        else:
            res2 += appointments[i]
            l2.append(appointments[i])
    if (res1 > res2):
        return l1, res1
    else:
        return l2, res2

In [29]:
mylist = [30, 40, 25, 50, 30, 20] 
print('The list is:', mylist, '\n') 
 
print('The best way to accept reservations in the given order is:', appointments(mylist)[0], 'with max duration of:', appointments(mylist)[1])

The list is: [30, 40, 25, 50, 30, 20] 

The best way to accept reservations in the given order is: [40, 50, 20] with max duration of: 110
